# Run 2015-2049

Memory errors for Xiaoya, so need to run one year at a time (this is probably an issue for a lot of computers)

In [ ]:
from climateforcing.utci import universal_thermal_climate_index, mean_radiant_temperature
from climateforcing.solar import cos_mean_solar_zenith_angle, modified_julian_date
import climateforcing
import cftime
import datetime
import numpy as np
import iris
import matplotlib.pyplot as pl
import warnings

from tqdm import tqdm  # stuff takes a while - let's find out how long

In [ ]:
cftime.__version__  # must be >= 1.4

In [ ]:
climateforcing.__version__  # must be >= 0.2.1

In [ ]:
# iris lazy-loads data, so I *think* it should be OK to keep these lines
path = '/nfs/b0110/Data/cmip6/HadGEM3-GC31-LL/ssp585/r1i1p1f3/3hr/'  # will need to change for JASMIN

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    tas_cube = iris.load_cube(path + 'tas/gn/tas_3hr_HadGEM3-GC31-LL_ssp585_r1i1p1f3_gn_201501010300-205001010000.nc')
    huss_cube = iris.load_cube(path + 'huss/gn/huss_3hr_HadGEM3-GC31-LL_ssp585_r1i1p1f3_gn_201501010300-205001010000.nc')
    rlds_cube = iris.load_cube(path + 'rlds/gn/rlds_3hr_HadGEM3-GC31-LL_ssp585_r1i1p1f3_gn_201501010130-204912302230.nc')
    rlus_cube = iris.load_cube(path + 'rlus/gn/rlus_3hr_HadGEM3-GC31-LL_ssp585_r1i1p1f3_gn_201501010130-204912302230.nc')
    rsds_cube = iris.load_cube(path + 'rsds/gn/rsds_3hr_HadGEM3-GC31-LL_ssp585_r1i1p1f3_gn_201501010130-204912302230.nc')
    rsus_cube = iris.load_cube(path + 'rsus/gn/rsus_3hr_HadGEM3-GC31-LL_ssp585_r1i1p1f3_gn_201501010130-204912302230.nc')
    rsdsdiff_cube = iris.load_cube(path + 'rsdsdiff/gn/rsdsdiff_3hr_HadGEM3-GC31-LL_ssp585_r1i1p1f3_gn_201501010130-204912302230.nc')
    uas_cube = iris.load_cube(path + 'uas/gn/uas_3hr_HadGEM3-GC31-LL_ssp585_r1i1p1f3_gn_201501010300-205001010000.nc')
    vas_cube = iris.load_cube(path + 'vas/gn/vas_3hr_HadGEM3-GC31-LL_ssp585_r1i1p1f3_gn_201501010300-205001010000.nc')

In [ ]:
lat = tas_cube.coord('latitude').points
lon = tas_cube.coord('longitude').points

lonmesh, latmesh = np.meshgrid(lon, lat)
lonmesh.shape

## Run everything in one cell

Avoid the memory errors, keep chunks to one year = 2880 time steps

In [ ]:
years = np.arange(2015, 2050)
nyears = len(years)
for iyear, year in tqdm(enumerate(years), total=nyears):  # tqdm give you a progress bar
    
    # get an array of zenith angles and lit fractions for 2100
    mjd = np.zeros(2880)
    first_day = cftime.datetime(year, 1, 1, 1, 30, calendar="360_day")
    for imjd in range(2880):
        mjd[imjd] = modified_julian_date(first_day + datetime.timedelta(hours=3*imjd))
    mean_cosz = np.ones((2880, 144, 192)) * np.nan
    lit = np.ones((2880, 144, 192)) * np.nan
    for i in range(2880):
        mean_cosz[i, ...], lit[i, ...] = cos_mean_solar_zenith_angle(mjd[i], 3, lat, lon)
        
    # calculate the mean radiant temperature. Here, we only extract one year at a time
    mrt = mean_radiant_temperature(
        {
            "rlds": rlds_cube[2880*iyear:2880*(iyear+1)].data,
            "rlus": rlus_cube[2880*iyear:2880*(iyear+1)].data,
            "rsdsdiff": rsdsdiff_cube[2880*iyear:2880*(iyear+1)].data,
            "rsus": rsus_cube[2880*iyear:2880*(iyear+1)].data,
            "rsds": rsds_cube[2880*iyear:2880*(iyear+1)].data,
        },
        angle_factor_down=0.5,
        angle_factor_up=0.5,
        absorption=0.7,
        emissivity=0.97,
        cos_zenith=mean_cosz,
        lit=lit
    )
    
    # wind - again one year at a time
    # uas and vas on different grids and should be regridded to the same grid as tas
    # I am not sure area weighted regridding makes sense for wind speed, so use linear
    uas_cube_regrid = uas_cube[2880*iyear:2880*(iyear+1)].regrid(tas_cube, iris.analysis.Linear())
    vas_cube_regrid = vas_cube[2880*iyear:2880*(iyear+1)].regrid(tas_cube, iris.analysis.Linear())
    wind = np.sqrt(uas_cube_regrid.data**2 + vas_cube_regrid.data**2)
    
    # calculate UTCI
    utci = universal_thermal_climate_index(
        {
            "tas": tas_cube[2880*iyear:2880*(iyear+1)].data,
            "sfcWind": wind,
            "huss": huss_cube[2880*iyear:2880*(iyear+1)].data
        },
        mrt
    )
    
    # make cubes out of output
    utci_cube = iris.cube.Cube(
        utci,
        units='K',
        var_name='utci',
        long_name='Universal Thermal Climate Index',
        dim_coords_and_dims=[
            (tas_cube[2880*iyear:2880*(iyear+1)].coord('time'), 0),
            (tas_cube.coord('latitude'), 1),
            (tas_cube.coord('longitude'), 2)
        ]
    )

    mrt_cube = iris.cube.Cube(
        mrt,
        units='K',
        var_name='mrt',
        long_name='Mean Radiant Temperature',
        dim_coords_and_dims=[
            (tas_cube[2880*iyear:2880*(iyear+1)].coord('time'), 0),
            (tas_cube.coord('latitude'), 1),
            (tas_cube.coord('longitude'), 2)
        ]
    )
    
    # Xiaoya (or anybody else following this!): change the output path to somewhere on your local machine
    # You might want to not save mrt to save space
    iris.save(utci_cube, '/nfs/b0110/Data/heatstress/utci_3hr_HadGEM3-GC31-LL_ssp585_r1i1p1f3_gn_%04d01010300-%04d01010000.nc' % (year, year+1))
    iris.save(utci_cube, '/nfs/b0110/Data/heatstress/mrt_3hr_HadGEM3-GC31-LL_ssp585_r1i1p1f3_gn_%04d01010300-%04d01010000.nc' % (year, year+1))